<a href="https://colab.research.google.com/github/alexauto/r_py_ewd/blob/main/Copy_of_book_neuro_net_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# indicate that you're running R code
# %%R

# Install if it's your first time to run
# install.packages("class")
# install.packages("data.table")
# install.packages("gmodels")
# install.packages("stringr")
# install.packages("dplyr")
# install.packages("tidyverse")

library(class)
# library(data.table)
library(gmodels)
library(stringr)
library(dplyr)
library(tidyverse)


In [ ]:
getwd()
# /content/sample_data/wisc_bc_data.csv

[1] "/content"

In [ ]:
library(class)
library(gmodels)

wbcd <- read.csv("/content/sample_data/wisc_bc_data.csv", stringsAsFactors = FALSE)

str(wbcd)
wbcd <- wbcd[-1]
table(wbcd$diagnosis)
wbcd$diagnosis<- factor(wbcd$diagnosis, levels = c("B", "M"),
                        labels = c("Benign", "Malignant"))

round(prop.table(table(wbcd$diagnosis)) * 100, digits = 1)
summary(wbcd[c("radius_mean", "area_mean", "smoothness_mean")])


normalize <- function(x) {
  return ((x - min(x)) / (max(x) - min(x)))
}
normalize(c(10, 20, 30, 40, 50))
#
wbcd_n <- as.data.frame(lapply(wbcd[2:31], normalize))
#
# Чтобы подтвердить, что преобразование было применено правильно,
# давайте посмотрим на сводную статистику одной переменной:
summary(wbcd_n$area_mean)

# Как и ожидалось, переменная area_mean,
# которая изначально находилась в диапазоне от 143,5 до 2501,0,
# теперь находится в диапазоне от 0 до 1.

wbcd_train <- wbcd_n[1:469, ]
wbcd_test <- wbcd_n[470:569, ]
#
# Когда мы создавали наши нормализованные обучающие и тестовые наборы данных, мы исключили
# целевую переменную, диагноз. Для обучения модели k-NN нам нужно будет сохранить
# эти метки классов в векторах факторов, разделенных между обучающим и тестовым наборами данных:
wbcd_train_labels <- wbcd[1:469, 1]
wbcd_test_labels <- wbcd[470:569, 1]
# Этот код берет фактор диагноза в первом столбце кадра данных wbcd и
# создает векторы wbcd_train_labels и wbcd_test_labels. Мы будем использовать их
# на следующих этапах обучения и оценки нашего классификатора.


wbcd_test_pred <- knn(train = wbcd_train, test = wbcd_test,
                      cl = wbcd_train_labels, k = 21)
# Функция knn() возвращает вектор-фактор предсказанных меток для каждого из
# примеров в тестовом наборе данных, который мы назначили wbcd_test_pred.

CrossTable(x = wbcd_test_labels, y = wbcd_test_pred,
           prop.chisq=FALSE)
#
# Проценты клеток в таблице указывают долю значений, которые попадают в четыре
# категории. Верхняя левая ячейка указывает на истинно отрицательные результаты. Эти 61 из 100 значений
# являются случаями, когда масса была доброкачественной, и алгоритм k-NN правильно идентифицировал ее
# как таковую. Нижняя правая ячейка указывает на истинно положительные результаты, когда классификатор
# и клинически определенная метка соглашаются, что масса является злокачественной. Всего 37 из
# 100 прогнозов были истинно положительными.
# Ячейки, попадающие на другую диагональ, содержат количество примеров, когда подход k-NN
# не согласуется с истинной меткой. Два примера в нижней левой ячейке
# являются ложноотрицательными результатами; в этом случае предсказанное значение было доброкачественным, но опухоль
# была на самом деле злокачественной. Ошибки в этом направлении могут быть чрезвычайно дорогостоящими, поскольку они
# могут заставить пациента поверить, что у него нет рака, но на самом деле болезнь может
# продолжать распространяться. Верхняя правая ячейка будет содержать ложноположительные результаты, если
# они были. Эти значения возникают, когда модель классифицирует массу как злокачественную,
# но на самом деле она была доброкачественной. Хотя такие ошибки менее опасны, чем ложноотрицательный результат, их также следует избегать, поскольку они могут привести к дополнительной финансовой
# нагрузке на систему здравоохранения или дополнительному стрессу для пациента, поскольку могут потребоваться дополнительные
# тесты или лечение.
# Если бы мы хотели, мы могли бы полностью исключить ложноотрицательные результаты,
# классифицируя каждую массу как злокачественную. Очевидно, что это нереалистичная
# стратегия. Тем не менее, она иллюстрирует тот факт, что прогнозирование включает в себя
# соблюдение баланса между ложноположительным и ложноотрицательным коэффициентами.
# В главе 10 «Оценка эффективности модели» вы узнаете о более
# сложных методах измерения точности прогнозирования, которые
# могут использоваться для определения мест, где можно оптимизировать частоту ошибок
# в зависимости от стоимости каждого типа ошибки.
# Всего 2 из 100, или 2 процента масс, были неправильно классифицированы с помощью подхода k-NN. Хотя точность в 98 процентов кажется впечатляющей для нескольких строк кода R,
# мы можем попробовать еще одну итерацию модели, чтобы посмотреть, сможем ли мы улучшить
# производительность и сократить количество значений, которые были неправильно классифицированы,
# особенно потому, что ошибки были опасными ложноотрицательными.
#
wbcd_z <- as.data.frame(scale(wbcd[-1]))
summary(wbcd_z$area_mean)
# Среднее значение стандартизированной переменной z-оценки всегда должно быть равно нулю, а диапазон
# должен быть достаточно компактным. Z-оценка больше 3 или меньше -3 указывает на
# крайне редкое значение. Учитывая это, преобразование, похоже, сработало.
# Как мы делали ранее, нам нужно разделить данные на обучающие и тестовые наборы, а
# затем классифицировать тестовые экземпляры с помощью функции knn(). Затем мы сравним
# предсказанные метки с фактическими метками с помощью CrossTable():
#
wbcd_train <- wbcd_z[1:469, ]
wbcd_test <- wbcd_z[470:569, ]
wbcd_train_labels <- wbcd[1:469, 1]
wbcd_test_labels <- wbcd[470:569, 1]
#
wbcd_test_pred <- knn(train = wbcd_train, test = wbcd_test,
                      cl = wbcd_train_labels, k = 21)

CrossTable(x = wbcd_test_labels, y = wbcd_test_pred,
           prop.chisq = FALSE)
#
# К сожалению, в следующей таблице результаты нашего нового преобразования показывают
# небольшое снижение точности. В случаях, когда мы правильно классифицировали 98 процентов примеров ранее, на этот раз мы классифицировали только 95 процентов правильно. Что еще хуже, мы не стали лучше классифицировать опасные ложноотрицательные результаты:
#
# Хотя классификатор никогда не был идеальным, подход 1-NN смог избежать
# некоторых ложных отрицательных результатов за счет добавления ложных положительных результатов. Однако важно
# иметь в виду, что было бы неразумно слишком тесно подгонять наш подход к
# нашим тестовым данным; в конце концов, другой набор из 100 записей пациентов, скорее всего, будет несколько
# отличаться от тех, которые использовались для измерения нашей эффективности.
# Если вам нужно быть уверенным, что обучающийся будет обобщать будущие
# данные, вы можете создать несколько наборов из 100 пациентов случайным образом и
# неоднократно повторно тестировать результат. Методы тщательной оценки
# производительности моделей машинного обучения будут обсуждаться
# далее в Главе 10, Оценка эффективности модели.

'data.frame':	569 obs. of  32 variables:
 $ id                     : int  842302 842517 84300903 84348301 84358402 843786 844359 84458202 844981 84501001 ...
 $ diagnosis              : chr  "M" "M" "M" "M" ...
 $ radius_mean            : num  18 20.6 19.7 11.4 20.3 ...
 $ texture_mean           : num  10.4 17.8 21.2 20.4 14.3 ...
 $ perimeter_mean         : num  122.8 132.9 130 77.6 135.1 ...
 $ area_mean              : num  1001 1326 1203 386 1297 ...
 $ smoothness_mean        : num  0.1184 0.0847 0.1096 0.1425 0.1003 ...
 $ compactness_mean       : num  0.2776 0.0786 0.1599 0.2839 0.1328 ...
 $ concavity_mean         : num  0.3001 0.0869 0.1974 0.2414 0.198 ...
 $ concave.points_mean    : num  0.1471 0.0702 0.1279 0.1052 0.1043 ...
 $ symmetry_mean          : num  0.242 0.181 0.207 0.26 0.181 ...
 $ fractal_dimension_mean : num  0.0787 0.0567 0.06 0.0974 0.0588 ...
 $ radius_se              : num  1.095 0.543 0.746 0.496 0.757 ...
 $ texture_se             : num  0.905 0.734 0.787 1


  B   M 
357 212 


   Benign Malignant 
     62.7      37.3 

  radius_mean       area_mean      smoothness_mean  
 Min.   : 6.981   Min.   : 143.5   Min.   :0.05263  
 1st Qu.:11.700   1st Qu.: 420.3   1st Qu.:0.08637  
 Median :13.370   Median : 551.1   Median :0.09587  
 Mean   :14.127   Mean   : 654.9   Mean   :0.09636  
 3rd Qu.:15.780   3rd Qu.: 782.7   3rd Qu.:0.10530  
 Max.   :28.110   Max.   :2501.0   Max.   :0.16340  

[1] 0.00 0.25 0.50 0.75 1.00

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.0000  0.1174  0.1729  0.2169  0.2711  1.0000 


 
   Cell Contents
|-------------------------|
|                       N |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  100 

 
                 | wbcd_test_pred 
wbcd_test_labels |    Benign | Malignant | Row Total | 
-----------------|-----------|-----------|-----------|
          Benign |        77 |         0 |        77 | 
                 |     1.000 |     0.000 |     0.770 | 
                 |     0.975 |     0.000 |           | 
                 |     0.770 |     0.000 |           | 
-----------------|-----------|-----------|-----------|
       Malignant |         2 |        21 |        23 | 
                 |     0.087 |     0.913 |     0.230 | 
                 |     0.025 |     1.000 |           | 
                 |     0.020 |     0.210 |           | 
-----------------|-----------|-----------|-----------|
    Column Total |        79 |        21 |       100 | 
           

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-1.4532 -0.6666 -0.2949  0.0000  0.3632  5.2459 


 
   Cell Contents
|-------------------------|
|                       N |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  100 

 
                 | wbcd_test_pred 
wbcd_test_labels |    Benign | Malignant | Row Total | 
-----------------|-----------|-----------|-----------|
          Benign |        77 |         0 |        77 | 
                 |     1.000 |     0.000 |     0.770 | 
                 |     0.975 |     0.000 |           | 
                 |     0.770 |     0.000 |           | 
-----------------|-----------|-----------|-----------|
       Malignant |         2 |        21 |        23 | 
                 |     0.087 |     0.913 |     0.230 | 
                 |     0.025 |     1.000 |           | 
                 |     0.020 |     0.210 |           | 
-----------------|-----------|-----------|-----------|
    Column Total |        79 |        21 |       100 | 
           